In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup as bs
import json

import re
from tqdm import tqdm

In [ ]:
# style 설정은 꼭 폰트설정 위에서 합니다.
# style 에 폰트 설정이 들어있으면 한글폰트가 초기화 되어 한글이 깨집니다.
import seaborn as sns
#나눔 고딕 적용
import matplotlib.font_manager
font_list = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
[matplotlib.font_manager.FontProperties(fname=font).get_name() for font in font_list if 'Nanum' in font]
\
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothicCoding')
\
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

# 음수 표시
matplotlib.rc("axes", unicode_minus = False)

# 한글을 unicode hex로 변환.
def keyword_to_unicode(keyword: 'str') -> str:
    keyword.encode('utf-8')
    regex = re.compile(r'\\x')
    keyword_unicode = re.sub(regex, '%', str(keyword.encode('utf-8'))[2:-1]).upper()
    return keyword_unicode

In [ ]:
#태그분석
tag_df = pd.read_csv('/content/tags(28).csv')

In [ ]:
tag_df.describe(include='object')

,keyword,related_tags
count,824,824
unique,27,177
top,슬픈,기분전환
freq,60,28


In [ ]:
related_tag_list = tag_df['related_tags'].unique().tolist()

In [ ]:
related_tag_list

['발라드',
 '감성',
 '잔잔한',
 '이별',
 '댄스',
 '사랑',
 '알앤비',
 '카페',
 '인디',
 '힙합',
 '기분전환',
 '휴식',
 '케이팝',
 '추억',
 '새벽',
 '드라이브',
 '최신가요',
 '오늘신곡모음',
 '겨울',
 '가을',
 '밤',
 '봄',
 '슬픔',
 '힐링',
 '설렘',
 '위로',
 '명곡',
 '비오는날',
 '매장',
 '출퇴근',
 '랩',
 '신나는',
 '국내힙합',
 '국힙',
 '감성힙합',
 '운동',
 '외힙',
 'HipHop',
 '매장음악',
 '트렌디',
 '스트레스',
 '그루브',
 '외국힙합',
 '클럽',
 '팝',
 '여행',
 'RnB',
 '인디음악',
 '여름',
 '어쿠스틱',
 '산책',
 '밴드',
 '분위기',
 '아이돌',
 'kpop',
 '걸그룹',
 'EDM',
 '노동요',
 'Pop',
 '가요',
 '헬스',
 '내적댄스',
 '파티',
 '청량',
 '커피',
 '재즈',
 '뉴에이지',
 '피아노',
 '연주곡',
 '시원한',
 '더위',
 '바다',
 '휴가',
 '여름밤',
 '청량한',
 '팝송',
 '노래방',
 '애창곡',
 '고음',
 '트로트',
 '히트곡',
 '2000년대',
 '회식',
 '코인노래방',
 '가창력',
 '인기곡',
 '남자',
 '노래방추천곡',
 '센치한',
 '비',
 '감성적인',
 '장마',
 '흐린날',
 '새벽감성',
 '혼자',
 '편안한',
 '주말',
 '공부',
 '연애',
 '회상',
 '달달',
 '썸',
 '지하철',
 '까페',
 '슬픈',
 '눈물',
 '우울',
 'OST',
 '그리움',
 '짝사랑',
 '우울한',
 '애절한',
 '아련한',
 '이별노래',
 '센치',
 '힘들때',
 '몽환',
 '외로움',
 '퇴근길',
 '기분좋은',
 '아침',
 '행복',
 '출근길',
 '상쾌한',
 '데이트',
 '일상',
 '여유',
 '고백',
 '헤어짐',
 '후회'

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

def keyword_to_unicode(keyword):
    return ''.join(['%{:02X}'.format(ord(c)) for c in keyword])

def get_playlist(keyword: str = '행복', page_no: int = 0, total: bool = True):
    keyword_unicode = keyword_to_unicode(keyword)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

    titles = []

    if total:
        playlist_tag_url = f'https://www.melon.com/dj/djfinder/djfinder_inform.htm?djSearchType=T&djSearchKeyword={keyword}'
        response_tag = requests.get(playlist_tag_url, headers=headers)
        html_tag = response_tag.text
        parse_tag = bs(html_tag, 'lxml')
        playlist_numbers_txt = parse_tag.find_all("p", "text")
        playlist_numbers = int(re.findall(r'[0-9]+', playlist_numbers_txt[0].text)[0])

        print(keyword, "키워드로 ", playlist_numbers, "전체 플레이리스트 수집을 시작합니다.")
        for page_no in range(playlist_numbers // 20 + 1):
            startIndex = page_no * 20 + 1
            playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?startIndex={startIndex}&pageSize=20&djSearchType=T&djSearchKeyword=%23{keyword_unicode}&orderBy=POP&pagingFlag=Y&tagSearchType=S"
            response = requests.get(playlist_url, headers=headers)
            html = response.text
            parse = bs(html, 'lxml')
            print(parse)
            titles_txt = parse.find_all(div.info, 'title')
            titles.extend(titles_txt)
            print(titles_txt)
            print("*", end="")
    else:
        print(keyword, "키워드로 ", page_no, "번 페이지 수집을 시작합니다.")
        startIndex = page_no * 20 + 1
        playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?startIndex={startIndex}&pageSize=20&djSearchType=T&djSearchKeyword=%23{keyword_unicode}&orderBy=POP&pagingFlag=Y&tagSearchType=S"
        response = requests.get(playlist_url, headers=headers)
        html = response.text
        parse = bs(html, 'lxml')
        titles_txt = parse.find_all("a", class_="ellipsis album_name")
        titles.extend(titles_txt)

    # make dataframe
    df = pd.DataFrame()
    plylstSeq_list = []
    title_list = []
    for title in titles:
        plylstSeq_list.append(re.findall(r'[0-9]+', title['href'])[0])
        title_list.append(title['title'])

    df['plylstSeq'] = plylstSeq_list
    df['title'] = title_list
    df['tag'] = keyword
    print("")
    print("complete.")

    return df


In [ ]:
body > div > ul > li:nth-child(2) > div.entry > div.info > a.ellipsis.album_name

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

def keyword_to_unicode(keyword):
    return ''.join(['%{:02X}'.format(ord(c)) for c in keyword])

def get_playlist(keyword: str = '행복', page_no: int = 0, total: bool = True):
    keyword_unicode = keyword_to_unicode(keyword)
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

    titles = []

    if total:
        playlist_tag_url = f'https://www.melon.com/dj/djfinder/djfinder_inform.htm?djSearchType=T&djSearchKeyword={keyword}'
        response_tag = requests.get(playlist_tag_url, headers=headers)
        html_tag = response_tag.text
        parse_tag = bs(html_tag, 'lxml')
        playlist_numbers_txt = parse_tag.find_all("p", class_="text")
        playlist_numbers = int(re.findall(r'[0-9]+', playlist_numbers_txt[0].text)[0])

        print(keyword, "키워드로 ", playlist_numbers, "전체 플레이리스트 수집을 시작합니다.")
        for page_no in range(playlist_numbers // 20 + 1):
            startIndex = page_no * 20 + 1
            playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?startIndex={startIndex}&pageSize=20&djSearchType=T&djSearchKeyword=%23{keyword_unicode}&orderBy=POP&pagingFlag=Y&tagSearchType=S"
            response = requests.get(playlist_url, headers=headers)
            html = response.text
            parse = bs(html, 'lxml')
            titles_txt  = parse.select(
            'body > div > ul > li:nth-child(1) > div.entry > div.info > a.ellipsis.album_name')
            print(titles_txt)
            print("*", end="")
    else:
        print(keyword, "키워드로 ", page_no, "번 페이지 수집을 시작합니다.")
        startIndex = page_no * 20 + 1
        playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?startIndex={startIndex}&pageSize=20&djSearchType=T&djSearchKeyword=%23{keyword_unicode}&orderBy=POP&pagingFlag=Y&tagSearchType=S"
        response = requests.get(playlist_url, headers=headers)
        html = response.text
        parse = bs(html, 'lxml')
        titles_txt = parse.find_all("a", class_="ellipsis album_name")
        for title in titles_txt:
            title_text = title.get_text(strip=True)
            titles.append(title_text)

    # make dataframe
    df = pd.DataFrame()
    plylstSeq_list = []
    title_list = []
    for title in titles:
        plylstSeq_list.append(re.findall(r'[0-9]+', title['href'])[0])
        title_list.append(title)

    df['plylstSeq'] = plylstSeq_list
    df['title'] = title_list
    df['tag'] = keyword
    print("")
    print("complete.")

    return df



In [ ]:
def get_playlist(keyword: str = '행복', page_no: int = 0, total: bool = True):
    # keyword to unicode
    keyword_unicode = keyword_to_unicode(keyword)
    # Headers information
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    }
    titles = []
    # get all playlists
    if total == True:
        # get tag numbers
        playlist_tag_url = f'https://www.melon.com/dj/djfinder/djfinder_inform.htm?djSearchType=T&djSearchKeyword={keyword}'
        # Request - Post
        response_tag = requests.get(playlist_tag_url, headers=headers)
        # beautifull suop loads
        html_tag = response_tag.text
        # parse text by beautifullsoup
        parse_tag = bs(html_tag, 'lxml')
        # find imformations
        playlist_numbers_txt = parse_tag.find_all("p", "text")
        playlist_numbers = int(re.findall(r'[0-9]+', playlist_numbers_txt[0].text)[0])

        print(keyword,"키워드로 ",playlist_numbers,"전체 플레이리스트 수집을 시작합니다.")
        for page_no in range(playlist_numbers//20 + 1):
            startIndex = page_no*20 + 1
            playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?startIndex={startIndex}&pageSize=20&djSearchType=T&djSearchKeyword=%23{keyword_unicode}&orderBy=POP&pagingFlag=Y&tagSearchType=S"
            # Request - Post
            response = requests.get(playlist_url, headers=headers)
            # beautifull suop loads
            html = response.text
            # parse text by beautifullsoup
            parse = bs(html, 'lxml')
            # find imformations
            # titles_txt = parse.find_all("a", "ellipsis album_name")
            titles_txt = parse.select("body > div > ul > li:nth-child(1) > div.entry > div.info > a.ellipsis.album_name")
            print(titles_txt)
            # titles extend
            titles.extend(titles_txt)
            print("*", end="")
    # get playlist page
    else:
            print(keyword,"키워드로 ",page_no,"번 페이지 수집을 시작합니다.")
            startIndex = page_no*20 + 1
            playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?startIndex={startIndex}&pageSize=20&djSearchType=T&djSearchKeyword=%23{keyword_unicode}&orderBy=POP&pagingFlag=Y&tagSearchType=S"
            # Request - Post
            response = requests.get(playlist_url, headers=headers)
            # beautifull suop loads
            html = response.text
            # parse text by beautifullsoup


            # body > div > ul > li:nth-child(1) > div.entry > div.info > a.ellipsis.album_name


            parse = bs(html, 'lxml')
            # find imformations
            titles_txt = parse.select("body > div > ul > li:nth-child(1) > div.entry > div.info > a.ellipsis.album_name")
            print(titles_txt)
            # titles extend
            titles.extend(titles_txt)

    # make dataframe
    df = pd.DataFrame()
    plylstSeq_list = []
    title_list = []
    for title_no in range(len(titles)):
        plylstSeq_list.append(re.findall(r'[0-9][0-9][0-9]+', titles[title_no]['href'])[0])
        title_list.append(titles[title_no]['title'][:-5])

    df['plylstSeq'] = plylstSeq_list
    df['title'] = title_list
    df['tag'] = keyword
    print("")
    print("complete.")
    return df

In [ ]:
# 사용예시
playlist_df = get_playlist(keyword='침착한', page_no=0, total=True)

침착한 키워드로  4 전체 플레이리스트 수집을 시작합니다.
*
complete.


In [ ]:
playlist_df

,plylstSeq,title,tag


In [ ]:
# 태그 리스트를 받아서 모든 플레이리스트를 csv 형태로 저장해주는 함수
def get_all_playlist(keyword_list: list = ['행복','사랑','슬픔']):
    playlist_df = pd.DataFrame()
    for idx, keyword in enumerate(tqdm(keyword_list)):
        print(idx, keyword)
        if(idx == 0):
            playlist_df = get_playlist(keyword, 0, True)
        else:
            playlist_df = pd.concat([playlist_df, get_playlist(keyword, 0, True)])
    playlist_df.to_csv(f'data/playlist({len(keyword_list)}).csv', index=False)
    return playlist_df

In [ ]:
all_playlist_df = get_all_playlist(related_tag_list)

In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

def keyword_to_unicode(keyword):
    return ''.join(['%{:02X}'.format(ord(c)) for c in keyword])

def get_playlist(keyword: str = '행복', page_no: int = 0, total: bool = True):
    # keyword to unicode
    keyword_unicode = keyword_to_unicode(keyword)
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
    }
    titles = []
    # get all playlists
    if total:
        # get tag numbers
        playlist_tag_url = f'https://www.melon.com/dj/djfinder/djfinder_inform.htm?djSearchType=T&djSearchKeyword={keyword}'
        # Request - Post
        response_tag = requests.get(playlist_tag_url, headers=headers)
        html_tag = response_tag.text
        parse_tag = bs(html_tag, 'lxml')
        playlist_numbers_txt = parse_tag.find_all("p", class_="text")
        playlist_numbers = int(re.findall(r'[0-9]+', playlist_numbers_txt[0].text)[0])

        print(keyword, "키워드로 ", playlist_numbers, "전체 플레이리스트 수집을 시작합니다.")
        for page_no in range(playlist_numbers // 20 + 1):
            startIndex = page_no * 20 + 1
            playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?djSearchType=T&djSearchKeyword={keyword_unicode}#params%5BdjSearchType%5D=T&params%5BdjSearchKeyword%5D={keyword_unicode}&params%5BorderBy%5D=POP&params%5BpagingFlag%5D=Y&params%5BtagSearchType%5D=S&po=pageObj&startIndex={startIndex}"
            response = requests.get(playlist_url, headers=headers)
            html = response.text
            parse = bs(html, 'lxml')
            titles_txt = parse.select("div.entry > div.info > a.ellipsis.album_name")
            titles.extend([title['title'] for title in titles_txt])
            print("*", end="")
    else:
        print(keyword, "키워드로 ", page_no, "번 페이지 수집을 시작합니다.")
        startIndex = page_no * 20 + 1
        playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?djSearchType=T&djSearchKeyword={keyword_unicode}#params%5BdjSearchType%5D=T&params%5BdjSearchKeyword%5D={keyword_unicode}&params%5BorderBy%5D=POP&params%5BpagingFlag%5D=Y&params%5BtagSearchType%5D=S&po=pageObj&startIndex={startIndex}"
        response = requests.get(playlist_url, headers=headers)
        html = response.text
        parse = bs(html, 'lxml')
        titles_txt = parse.select("div.entry > div.info > a.ellipsis.album_name")
        titles.extend([title['title'] for title in titles_txt])

    # make dataframe
    df = pd.DataFrame()
    plylstSeq_list = []
    title_list = []
    for title in titles:
        plylstSeq_list.append(re.findall(r'[0-9]+', title['href'])[0])
        title_list.append(title)

    df['plylstSeq'] = plylstSeq_list
    df['title'] = title_list
    df['tag'] = keyword
    print("")
    print("complete.")
    return df


In [ ]:
# 사용예시
playlist_df = get_playlist(keyword='침착한', page_no=0, total=True)

침착한 키워드로  4 전체 플레이리스트 수집을 시작합니다.
*
complete.


In [ ]:
playlist_df

,plylstSeq,title,tag


In [ ]:
https://www.melon.com/dj/djfinder/djfinder_inform.htm?djSearchType=T&djSearchKeyword={keyword_unicode}#params%5BdjSearchType%5D=T&params%5BdjSearchKeyword%5D={keyword_unicode}&params%5BorderBy%5D=POP&params%5BpagingFlag%5D=Y&params%5BtagSearchType%5D=S&po=pageObj&startIndex={startIndex}

In [ ]:
 def get_playlist(keyword: str = '행복', page_no: int = 0, total: bool = True):
    # keyword to unicode
    keyword_unicode = keyword_to_unicode(keyword)
    # Headers information
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36',
    }
    titles = []
    # get all playlists
    if total == True:
        # get tag numbers
        playlist_tag_url = f'https://www.melon.com/dj/djfinder/djfinder_inform.htm?djSearchType=T&djSearchKeyword={keyword}'
        # Request - Post
        response_tag = requests.get(playlist_tag_url, headers=headers)
        # beautifull suop loads
        html_tag = response_tag.text
        # parse text by beautifullsoup
        parse_tag = bs(html_tag, 'lxml')
        # find imformations
        playlist_numbers_txt = parse_tag.find_all("p", "text")
        playlist_numbers = int(re.findall(r'[0-9]+', playlist_numbers_txt[0].text)[0])

        print(keyword,"키워드로 ",playlist_numbers,"전체 플레이리스트 수집을 시작합니다.")
        for page_no in range(playlist_numbers//20 + 1):
            startIndex = page_no*20 + 1
            playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?startIndex={startIndex}&pageSize=20&djSearchType=T&djSearchKeyword=%23{keyword_unicode}&orderBy=POP&pagingFlag=Y&tagSearchType=S"
            # Request - Post
            response = requests.get(playlist_url, headers=headers)
            # beautifull suop loads
            html = response.text
            # parse text by beautifullsoup
            parse = bs(html, 'lxml')
            # find imformations
            titles_txt = parse.find_all("a", "ellipsis album_name")
            # titles extend
            titles.extend(titles_txt)
            print("*", end="")
    # get playlist page
    else:
            print(keyword,"키워드로 ",page_no,"번 페이지 수집을 시작합니다.")
            startIndex = page_no*20 + 1
            playlist_url = f"https://www.melon.com/dj/djfinder/djfinder_inform.htm?startIndex={startIndex}&pageSize=20&djSearchType=T&djSearchKeyword=%23{keyword_unicode}&orderBy=POP&pagingFlag=Y&tagSearchType=S"
            # Request - Post
            response = requests.get(playlist_url, headers=headers)
            # beautifull suop loads
            html = response.text
            # parse text by beautifullsoup
            parse = bs(html, 'lxml')
            # find imformations
            titles_txt = parse.find_all("a", "ellipsis album_name")
            # titles extend
            titles.extend(titles_txt)

    # make dataframe
    df = pd.DataFrame()
    plylstSeq_list = []
    title_list = []
    for title_no in range(len(titles)):
        plylstSeq_list.append(re.findall(r'[0-9][0-9][0-9]+', titles[title_no]['href'])[0])
        title_list.append(titles[title_no]['title'][:-5])

    df['plylstSeq'] = plylstSeq_list
    df['title'] = title_list
    df['tag'] = keyword
    print("")
    print("complete.")
    return df